In [1]:
import pandas as pd
import os as os
import numpy as np
import talib as talib
import json as json
from datetime import datetime, timedelta, date, time
from os import walk
pd.options.mode.chained_assignment = None

In [2]:
"""
1.0 传叁
file1 = request.form.get('file1') # 文件1
file2 = request.form.get('file2') # 文件2
direction = request.form.get('direction') # 方向
vol = float(request.form.get('vol'))  # 买卖波幅
vol2 = float(request.form.get('vol2'))  # 止蚀波幅
cutloss = float(request.form.get('cutLoss'))  # 止蚀
is_adjust = False if request.form.get('adjection')=='false' else True # 调整
minutes = int(request.form.get('minutes')) # 预测x分钟后
"""

# 1.0 传叁
file1 = '20201211-111436'
file2 = 't5-0921-1113-e12-u512-w50-lr0.0005'
direction = 'long' # 方向
vol = 0.35 # 买卖波幅
vol2 = 0.0 # 止蚀波幅
cutloss = 1.50 # 止蚀
is_adjust = False # 调整
minutes = 5 # 预测x分钟后

# 1.1 数据集
path1 = os.path.abspath(os.path.join('data', 'nq', 'prediction', file1, file2+'.csv'))
df1 = pd.read_csv(path1)
df1.udate = pd.to_datetime(df1.udate)
df1.index = pd.to_datetime(df1.udate)
df1

,udate,High,Low,Open,Close,Volume,t1,t2,t3,t4,t5
udate,,,,,,,,,,,
2020-09-21 17:00:00,2020-09-21 17:00:00,10966.25,10970.00,10956.0,10960.25,676.0,NaN,NaN,NaN,NaN,NaN
2020-09-21 17:01:00,2020-09-21 17:01:00,10960.25,10971.75,10960.0,10971.00,363.0,NaN,NaN,NaN,NaN,NaN
2020-09-21 17:02:00,2020-09-21 17:02:00,10971.00,10974.00,10968.0,10968.75,216.0,NaN,NaN,NaN,NaN,NaN
2020-09-21 17:03:00,2020-09-21 17:03:00,10968.75,10970.50,10965.0,10969.75,199.0,NaN,NaN,NaN,NaN,NaN
2020-09-21 17:04:00,2020-09-21 17:04:00,10969.75,10976.00,10969.5,10976.00,97.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2020-11-13 15:48:00,2020-11-13 15:48:00,11946.50,11946.50,11944.5,11944.50,21.0,11956.374919,11950.640173,11951.051937,11955.301104,11954.833929
2020-11-13 15:49:00,2020-11-13 15:49:00,11944.50,11945.25,11943.0,11944.25,38.0,11954.114717,11948.420122,11948.833137,11953.134838,11952.655530
2020-11-13 15:50:00,2020-11-13 15:50:00,11944.25,11946.50,11943.0,11945.00,107.0,11952.438328,11946.775691,11947.182952,11951.527555,11951.025733


In [3]:
# 2.0 数据集2
df2 = df1.copy(deep=True)

highs = np.array(df2['High'], dtype='float')
lows = np.array(df2['Low'], dtype='float')
opens = np.array(df2['Open'], dtype='float')
closes = np.array(df2['Close'], dtype='float')
vols = np.array(df2['Volume'], dtype='float')
df2['atr'] = talib.ATR(highs, lows, closes, timeperiod=14*2)

df2['last1'] = df2['Close'].shift(periods=-1) # 上笔数据 最后价
df2['adjust'] = df2['Close'] - df2['last1'] # 前1分钟 结算价 波幅

key2 = 't'+str(minutes)
if not is_adjust:
    df2['plast'] = df2[key2] # 预测 5分钟后的 波幅 
    df2['plast1'] = df2['t1'] # 预测 1分钟后的 波幅
else:
    df2['plast'] = df2[key2] + df2['adjust'] # 已调整
    df2['plast1'] = df2['t1'] + df2['adjust'] # 已调整

df2['pchange'] = (df2['plast'] - df2['Close']) / df2['Close']*100 # 预测 5分钟后的 波幅率
df2['pchange1'] = (df2['plast1'] - df2['Close']) / df2['Close']*100 # 预测 1分钟后的 波幅率

df2['real(t+1)'] = df2['Close'].shift(periods=-1)
df2['real(t+5)'] = df2['Close'].shift(periods=-5)

df2

,udate,High,Low,Open,Close,Volume,t1,t2,t3,t4,t5,atr,last1,adjust,plast,plast1,pchange,pchange1,real(t+1),real(t+5)
udate,,,,,,,,,,,,,,,,,,,,
2020-09-21 17:00:00,2020-09-21 17:00:00,10966.25,10970.00,10956.0,10960.25,676.0,NaN,NaN,NaN,NaN,NaN,NaN,10971.00,-10.75,NaN,NaN,NaN,NaN,10971.00,10973.00
2020-09-21 17:01:00,2020-09-21 17:01:00,10960.25,10971.75,10960.0,10971.00,363.0,NaN,NaN,NaN,NaN,NaN,NaN,10968.75,2.25,NaN,NaN,NaN,NaN,10968.75,10972.75
2020-09-21 17:02:00,2020-09-21 17:02:00,10971.00,10974.00,10968.0,10968.75,216.0,NaN,NaN,NaN,NaN,NaN,NaN,10969.75,-1.00,NaN,NaN,NaN,NaN,10969.75,10975.25
2020-09-21 17:03:00,2020-09-21 17:03:00,10968.75,10970.50,10965.0,10969.75,199.0,NaN,NaN,NaN,NaN,NaN,NaN,10976.00,-6.25,NaN,NaN,NaN,NaN,10976.00,10974.50
2020-09-21 17:04:00,2020-09-21 17:04:00,10969.75,10976.00,10969.5,10976.00,97.0,NaN,NaN,NaN,NaN,NaN,NaN,10973.00,3.00,NaN,NaN,NaN,NaN,10973.00,10979.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-13 15:48:00,2020-11-13 15:48:00,11946.50,11946.50,11944.5,11944.50,21.0,11956.374919,11950.640173,11951.051937,11955.301104,11954.833929,2.613252,11944.25,0.25,11954.833929,11956.374919,0.086516,0.099417,11944.25,NaN
2020-11-13 15:49:00,2020-11-13 15:49:00,11944.50,11945.25,11943.0,11944.25,38.0,11954.114717,11948.420122,11948.833137,11953.134838,11952.655530,2.546707,11945.00,-0.75,11952.655530,11954.114717,0.070373,0.082590,11945.00,NaN
2020-11-13 15:50:00,2020-11-13 15:50:00,11944.25,11946.50,11943.0,11945.00,107.0,11952.438328,11946.775691,11947.182952,11951.527555,11951.025733,2.536111,11946.25,-1.25,11951.025733,11952.438328,0.050446,0.062271,11946.25,NaN


In [4]:
# 3.0 回测
df4 = pd.DataFrame(columns = ['udate', 'action', 'cash', 'profit', 'message'])
holding = False # 是否持有仓位
top_profit_price = 0 # 买入点数
fee = 2.1 # 交易费
cash = 0 # 资金流

def is_trigger(v2, vol):
    if direction == 'long':
        return (v2['pchange'] >= vol)
    elif direction == 'short':
        return (v2['pchange'] <= -vol)

def is_stop_profit(v2, top_profit_price, direction):
    if direction == 'long':
        return (abs(v1['Close']-top_profit_price) >= abs(cutloss*v1['atr']))
    elif direction == 'short':
        return (abs(top_profit_price-v1['Close']) >= abs(cutloss*v1['atr']))
    
def is_cut_loss(v2, vol2):
    if direction == 'long':
        return (v2['pchange1'] <= vol2)
    elif direction == 'short':
        return (v2['pchange1'] >= vol2)

df3 = df2.copy(deep=True)
for k1, v1 in df3.iterrows():
    if v1['udate'].hour <= 14 or v1['udate'].hour >= 19:
        # 时间
        cur_time, close_time = time(v1.udate.hour, v1.udate.minute, 0), time(14, 59, 0)
        is_over_night = (cur_time == close_time) # 过夜
        # 3.2 买入
        if is_trigger(v1, vol) and not is_over_night and not holding:
            holding = True
            cost1 = v1['Close']
            top_profit_price = v1['Close']
            action = 'buy' if direction == 'long' else 'sell'
            start_hold_time = k1
            df4 = df4.append({'udate': k1, 'action': action, 'cash': np.nan, 'profit': np.nan, 'hold time': np.nan, 'message': np.nan}, ignore_index=True)
            continue
        # 3.3 卖出
        cut_loss2 = is_cut_loss(v1, vol2) # 止蚀
        stop_profit2 = is_stop_profit(v1, top_profit_price, direction) # 止盈
        if (cut_loss2 or stop_profit2 or is_over_night) and holding:
            holding = False
            # 3.4 持有时长
            mins_diff = (k1 - start_hold_time).total_seconds() / 60
            # 3.5 盈亏
            if direction == 'long':
                profit2 = (v1['Close']-cost1)*20 - (fee*2)
            elif direction == 'short':
                profit2 = (cost1-v1['Close'])*20 - (fee*2)
            if profit2 > 0:
                profit3 = '<span class="text-danger">+'+str(round(profit2, 1))+'</span>'
            elif profit2 <= 0:
                profit3 = '<span class="text-success">'+str(round(profit2, 1))+'</span>'
            message = '('+str(cost1)+' - '+str(v1['Close'])+')*20 - ('+str(fee)+'*2) = '+profit3
            # 3.6 长仓
            if direction == 'long' and stop_profit2:
                action = 'sell'
            elif direction == 'long' and cut_loss2:
                action = 'cut'
            elif direction == 'long' and is_over_night:
                action = 'cut overnight'
            # 3.7 短仓
            elif direction == 'short' and stop_profit2:
                action = 'buy'
            elif direction == 'short' and cut_loss2:
                action = 'cut'
            elif direction == 'short' and is_over_night:
                action = 'cut overnight'
            # 3.8 资金流
            cash = cash + profit2
            
            df4 = df4.append({'udate': k1, 'action': action, 'cash': cash, 'profit': profit2, 'hold time': mins_diff, 'message': message}, ignore_index=True)
            continue

df4.index = df4['udate']
df4.drop(['udate'], axis=1, inplace=True)
df4

,action,cash,profit,message,hold time
udate,,,,,
2020-10-01 23:55:00,buy,NaN,NaN,NaN,NaN
2020-10-01 23:57:00,sell,-74.2,-74.2,(11489.5 - 11486.0)*20 - (2.1*2) = <span class...,2.0
2020-10-06 13:49:00,buy,NaN,NaN,NaN,NaN
2020-10-06 13:50:00,sell,-873.4,-799.2,(11454.5 - 11414.75)*20 - (2.1*2) = <span clas...,1.0
2020-10-06 13:51:00,buy,NaN,NaN,NaN,NaN
2020-10-06 13:52:00,sell,-1177.6,-304.2,(11398.5 - 11383.5)*20 - (2.1*2) = <span class...,1.0
2020-10-06 13:54:00,buy,NaN,NaN,NaN,NaN
2020-10-06 13:55:00,sell,-1556.8,-379.2,(11336.0 - 11317.25)*20 - (2.1*2) = <span clas...,1.0
2020-10-06 13:56:00,buy,NaN,NaN,NaN,NaN


In [5]:
# 4.0 清理数据集
df5 = pd.concat([df2, df4], axis=1)
drop_list_5 = ['High', 'Low', 'Open', 'Volume', 't2', 't3', 't4', 'last1']
df5.drop(drop_list_5, axis=1, inplace=True)
df5

,udate,Close,t1,t5,atr,adjust,plast,plast1,pchange,pchange1,real(t+1),real(t+5),action,cash,profit,message,hold time
udate,,,,,,,,,,,,,,,,,
2020-09-21 17:00:00,2020-09-21 17:00:00,10960.25,NaN,NaN,NaN,-10.75,NaN,NaN,NaN,NaN,10971.00,10973.00,NaN,NaN,NaN,NaN,NaN
2020-09-21 17:01:00,2020-09-21 17:01:00,10971.00,NaN,NaN,NaN,2.25,NaN,NaN,NaN,NaN,10968.75,10972.75,NaN,NaN,NaN,NaN,NaN
2020-09-21 17:02:00,2020-09-21 17:02:00,10968.75,NaN,NaN,NaN,-1.00,NaN,NaN,NaN,NaN,10969.75,10975.25,NaN,NaN,NaN,NaN,NaN
2020-09-21 17:03:00,2020-09-21 17:03:00,10969.75,NaN,NaN,NaN,-6.25,NaN,NaN,NaN,NaN,10976.00,10974.50,NaN,NaN,NaN,NaN,NaN
2020-09-21 17:04:00,2020-09-21 17:04:00,10976.00,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN,10973.00,10979.75,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-13 15:48:00,2020-11-13 15:48:00,11944.50,11956.374919,11954.833929,2.613252,0.25,11954.833929,11956.374919,0.086516,0.099417,11944.25,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-13 15:49:00,2020-11-13 15:49:00,11944.25,11954.114717,11952.655530,2.546707,-0.75,11952.655530,11954.114717,0.070373,0.082590,11945.00,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-13 15:50:00,2020-11-13 15:50:00,11945.00,11952.438328,11951.025733,2.536111,-1.25,11951.025733,11952.438328,0.050446,0.062271,11946.25,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# 5.0 渲染1
df6 = df5.loc[(df5['action']=='buy') | (df5['action']=='sell') | (df5['action']=='cut') | (df5['action']=='cut overnight')]
df6.columns = ['udate', 'last', 'predict1(T+1)', 'predict1(T+5)', 'atr', 'shift', 'predict2(T+1)', 'predict2(T+5)', 'p-percent1', 'p-percent5', 'real(t+1)', 'real(t+5)', 
               'action', 'cash', 'profit', 'message', 'hold time']
df6['cash'] = df6['cash'].round(4)
df6['profit'] = df6['profit'].round(2)
# 5.1 渲染2
if direction == 'long':
    no_trade = df6.loc[(df6['action']=='buy')].shape[0]
elif direction == 'short':
    no_trade = df6.loc[(df6['action']=='sell')].shape[0]
total_profit = df6.iloc[-1]['cash'].round(2)
avg_profit = df6['profit'].mean().round(2)
max_win = df6['profit'].max().round(2)
max_loss = df6['profit'].min().round(2)
no_win = int(np.sum(df6['profit'] > 0))
no_loss = int(np.sum(df6['profit'] <= 0))
no_cut_loss = int(np.sum(df6['action'] == 'cut')) + int(np.sum(df6['action'] == 'cut overnight'))
no_total_day = len(df5.groupby(df5['udate'].dt.date).size())
no_trade_day = len(df5.groupby(df6['udate'].dt.date).size())
avg_no_trade = round(no_trade/no_total_day, 2)
hold_time_max = df6['hold time'].max().round(0)
hold_time_min = df6['hold time'].min().round(0)
hold_time_avg = (df6['hold time'].sum()/no_trade).round(0)

# 5.2 加颜色
for k5, v5 in df6.iterrows():
    # 5.3 profit
    if df6.loc[k5, 'profit'] > 0:
        df6.loc[k5, 'profit'] = '<font class="text-danger">+'+str(df6.loc[k5, 'profit'])+'</font>'
    elif df6.loc[k5, 'profit'] <= 0:
        df6.loc[k5, 'profit'] = '<font class="text-success">'+str(df6.loc[k5, 'profit'])+'</font>'
    # 5.4 cash
    if df6.loc[k5, 'cash'] > 0:
        df6.loc[k5, 'cash'] = '<font class="text-danger">+'+str(df6.loc[k5, 'cash'])+'</font>'
    elif df6.loc[k5, 'cash'] <= 0:
        df6.loc[k5, 'cash'] = '<font class="text-success">'+str(df6.loc[k5, 'cash'])+'</font>'
html = df6.to_html(classes='table table-sm table-striped', index=False, escape=False, border=0, justify='left').replace('border="1"', 'border="0"').replace('NaN', '')
data4 = {'total_profit': total_profit, 'avg_profit': avg_profit, 'max_win': max_win, 'max_loss': max_loss,
         'no_trade': no_trade, 'no_win': no_win, 'no_loss': no_loss, 'no_cut_loss': no_cut_loss, 'avg_no_trade': avg_no_trade, 'no_total_day': no_total_day, 
         'hold_time_max': hold_time_max, 'hold_time_min': hold_time_min, 'hold_time_avg': hold_time_avg,
         'html': html}
data5 = json.dumps(data4)

del data4['html']
print(data4)
df6

{'total_profit': -349.6, 'avg_profit': -26.89, 'max_win': 650.8, 'max_loss': -799.2, 'no_trade': 13, 'no_win': 5, 'no_loss': 8, 'no_cut_loss': 0, 'avg_no_trade': 0.33, 'no_total_day': 39, 'hold_time_max': 3.0, 'hold_time_min': 1.0, 'hold_time_avg': 1.0}


,udate,last,predict1(T+1),predict1(T+5),atr,shift,predict2(T+1),predict2(T+5),p-percent1,p-percent5,real(t+1),real(t+5),action,cash,profit,message,hold time
udate,,,,,,,,,,,,,,,,,
2020-10-01 23:55:00,2020-10-01 23:55:00,11489.50,11534.474991,11533.479112,1.580401,2.50,11533.479112,11534.474991,0.382777,0.391444,11487.00,11462.00,buy,NaN,NaN,NaN,NaN
2020-10-01 23:57:00,2020-10-01 23:57:00,11486.00,11510.150920,11509.208495,2.048613,-1.50,11509.208495,11510.150920,0.202059,0.210264,11487.50,11431.25,sell,"<font class=""text-success"">-74.2</font>","<font class=""text-success"">-74.2</font>",(11489.5 - 11486.0)*20 - (2.1*2) = <span class...,2.0
2020-10-06 13:49:00,2020-10-06 13:49:00,11454.50,11501.292550,11500.491307,3.707513,39.75,11500.491307,11501.292550,0.401513,0.408508,11414.75,11336.00,buy,NaN,NaN,NaN,NaN
2020-10-06 13:50:00,2020-10-06 13:50:00,11414.75,11482.251850,11481.775491,3.628674,16.25,11481.775491,11482.251850,0.587183,0.591356,11398.50,11317.25,sell,"<font class=""text-success"">-873.4</font>","<font class=""text-success"">-799.2</font>",(11454.5 - 11414.75)*20 - (2.1*2) = <span clas...,1.0
2020-10-06 13:51:00,2020-10-06 13:51:00,11398.50,11461.506871,11461.221055,3.543721,15.00,11461.221055,11461.506871,0.550257,0.552765,11383.50,11311.50,buy,NaN,NaN,NaN,NaN
2020-10-06 13:52:00,2020-10-06 13:52:00,11383.50,11441.222776,11440.969443,3.452874,-9.75,11440.969443,11441.222776,0.504849,0.507074,11393.25,11342.50,sell,"<font class=""text-success"">-1177.6</font>","<font class=""text-success"">-304.2</font>",(11398.5 - 11383.5)*20 - (2.1*2) = <span class...,1.0
2020-10-06 13:54:00,2020-10-06 13:54:00,11336.00,11404.949881,11404.445132,4.296422,18.75,11404.445132,11404.949881,0.603786,0.608238,11317.25,11343.50,buy,NaN,NaN,NaN,NaN
2020-10-06 13:55:00,2020-10-06 13:55:00,11317.25,11383.092841,11382.497488,4.607264,5.75,11382.497488,11383.092841,0.576531,0.581792,11311.50,11324.25,sell,"<font class=""text-success"">-1556.8</font>","<font class=""text-success"">-379.2</font>",(11336.0 - 11317.25)*20 - (2.1*2) = <span clas...,1.0
2020-10-06 13:56:00,2020-10-06 13:56:00,11311.50,11361.354412,11360.629578,4.451648,-31.00,11360.629578,11361.354412,0.434333,0.440741,11342.50,11317.75,buy,NaN,NaN,NaN,NaN
